In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import statistics as st
import datetime

# Change Directory to file location

In [2]:
os.chdir(r'C:\Users\kjsdr\Documents\GitHub\foster-app\sample-data\Sadhanand')

# Read file content into Pandas DataFrame

In [3]:
dataset=pd.read_excel('internbuddy_data_v1.xlsx')

# Have a look at the Data

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 24 columns):
Application_ID                392 non-null object
Current City                  392 non-null object
Python (out of 3)             392 non-null int64
R Programming (out of 3)      392 non-null int64
Deep Learning (out of 3)      392 non-null int64
PHP (out of 3)                392 non-null int64
MySQL (out of 3)              392 non-null int64
HTML (out of 3)               392 non-null int64
CSS (out of 3)                392 non-null int64
JavaScript (out of 3)         392 non-null int64
Unnamed: 10                   0 non-null float64
AJAX (out of 3)               392 non-null int64
Bootstrap (out of 3)          392 non-null int64
MongoDB (out of 3)            392 non-null int64
Node.js (out of 3)            392 non-null int64
ReactJS (out of 3)            392 non-null int64
Other skills                  378 non-null object
Degree                        380 non-null object
Stream   

In [5]:
print(dataset.columns.tolist())

['Application_ID', 'Current City', 'Python (out of 3)', 'R Programming (out of 3)', 'Deep Learning (out of 3)', 'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)', 'CSS (out of 3)', 'JavaScript (out of 3)', 'Unnamed: 10', 'AJAX (out of 3)', 'Bootstrap (out of 3)', 'MongoDB (out of 3)', 'Node.js (out of 3)', 'ReactJS (out of 3)', 'Other skills', 'Degree', 'Stream', 'Current Year Of Graduation', 'Performance_PG', 'Performance_UG', 'Performance_12', 'Performance_10']


In [6]:
del dataset['Unnamed: 10']

In [7]:
print(dataset.columns.tolist())

['Application_ID', 'Current City', 'Python (out of 3)', 'R Programming (out of 3)', 'Deep Learning (out of 3)', 'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)', 'CSS (out of 3)', 'JavaScript (out of 3)', 'AJAX (out of 3)', 'Bootstrap (out of 3)', 'MongoDB (out of 3)', 'Node.js (out of 3)', 'ReactJS (out of 3)', 'Other skills', 'Degree', 'Stream', 'Current Year Of Graduation', 'Performance_PG', 'Performance_UG', 'Performance_12', 'Performance_10']


# Looks like there are some null values for some of the columns.Need to Check whether all null values have been captured(those marked as N/A in excel file as well).check row for application ID ML16

In [8]:
id_filt=(dataset['Application_ID']=='ML0016')
dataset.loc[id_filt,['Performance_UG','Performance_12']]

,Performance_UG,Performance_12
15,8.00/10,NaN


# check empty space capture.check row for ML0025

In [9]:
id_filt=(dataset['Application_ID']=='ML0025')
dataset.loc[id_filt,['Stream']]

,Stream
24,NaN


# check percentage of NaN values for each column

In [10]:
dataset.isna().mean().round(4) * 100

Application_ID                 0.00
Current City                   0.00
Python (out of 3)              0.00
R Programming (out of 3)       0.00
Deep Learning (out of 3)       0.00
PHP (out of 3)                 0.00
MySQL (out of 3)               0.00
HTML (out of 3)                0.00
CSS (out of 3)                 0.00
JavaScript (out of 3)          0.00
AJAX (out of 3)                0.00
Bootstrap (out of 3)           0.00
MongoDB (out of 3)             0.00
Node.js (out of 3)             0.00
ReactJS (out of 3)             0.00
Other skills                   3.57
Degree                         3.06
Stream                         4.59
Current Year Of Graduation     0.00
Performance_PG                73.72
Performance_UG                13.78
Performance_12                41.58
Performance_10                45.15
dtype: float64

# even though there is a high percentage of missing values in Performance_ metrics,we are not going to remove those columns,since we deem it important for judging a candidate.

# going to impute missing values with model based imputation since it aint appropriate to take the mean/median here since large part of the columns have missing values.instead,imputation is done by taking those rows that have values for columns as training set,and predicting the values for missing data.

# Since KNN requires other values to be numerical,we will do some feature engineering to convert columns into definable metrics and then do knn imputation.

# Degree and stream are categorical variables.lets us find the set of unique values first

In [11]:
dataset.Degree.unique()

array(['Bachelor of Science (B.Sc)', 'Bachelor of Technology (B.Tech)',
       'Master of Science (M.Sc)', 'Bachelor of Engineering (B.E)', nan,
       'PG Diploma in Data Science', 'Post Graduate Programme (PGP)',
       'Post Graduate Diploma', 'Master of Computer Applications (MCA)',
       'Master of Technology (M.Tech)', 'Executive MBA', 'MBA',
       'B.Tech (Hons.)',
       'Post Graduate Diploma in Big Data Analytics (PG-DBDA)',
       'Bachelor of Commerce (B.Com)',
       'Bachelor of Engineering (B.E) (Hons.)',
       'Bachelor of Computer Applications (BCA)', 'Integrated M.Sc.',
       'Bachelor of Business Management (B.B.M.)',
       'Bachelor of Commerce (B.Com) (Hons.)',
       'Post Graduate Diploma in Management (P.G.D.M.)',
       'Integrated B.Tech & M.Tech', 'Master of Science (M.S.)',
       'Post Graduate Program in Business Analytics',
       'Bachelor of Computer Science (B.C.S.)', 'Integrated B.Tech',
       'Master of Science (M.Sc) (Hons.)', 'Integrated M.Te

# there are too many values .high cardinality.need to club some of the values=>Bachelor program,master program,Integrated program,Post Graduate Program

In [12]:
lb_Degree = LabelEncoder()

In [13]:
dataset["Degree_code"] = lb_Degree.fit_transform(list(dataset["Degree"]))

In [14]:
dataset[["Degree", "Degree_code"]].head(11)

,Degree,Degree_code
0,Bachelor of Science (B.Sc),11
1,Bachelor of Technology (B.Tech),12
2,Master of Science (M.Sc),22
3,Bachelor of Engineering (B.E),9
4,Bachelor of Technology (B.Tech),12
5,NaN,31
6,NaN,31
7,PG Diploma in Data Science,25
8,Post Graduate Programme (PGP),30
9,Post Graduate Programme (PGP),30


In [15]:
array_degree_code=dataset.Degree_code.unique()
array_degree_code

array([11, 12, 22,  9, 31, 25, 30, 26, 20, 24, 13, 19,  1, 27,  5, 10,  7,
       17,  4,  6, 28, 16, 21, 29,  8, 15, 23, 18,  0,  3,  2, 14],
      dtype=int64)

In [16]:
array_degree=dataset.Degree.unique()
array_degree

array(['Bachelor of Science (B.Sc)', 'Bachelor of Technology (B.Tech)',
       'Master of Science (M.Sc)', 'Bachelor of Engineering (B.E)', nan,
       'PG Diploma in Data Science', 'Post Graduate Programme (PGP)',
       'Post Graduate Diploma', 'Master of Computer Applications (MCA)',
       'Master of Technology (M.Tech)', 'Executive MBA', 'MBA',
       'B.Tech (Hons.)',
       'Post Graduate Diploma in Big Data Analytics (PG-DBDA)',
       'Bachelor of Commerce (B.Com)',
       'Bachelor of Engineering (B.E) (Hons.)',
       'Bachelor of Computer Applications (BCA)', 'Integrated M.Sc.',
       'Bachelor of Business Management (B.B.M.)',
       'Bachelor of Commerce (B.Com) (Hons.)',
       'Post Graduate Diploma in Management (P.G.D.M.)',
       'Integrated B.Tech & M.Tech', 'Master of Science (M.S.)',
       'Post Graduate Program in Business Analytics',
       'Bachelor of Computer Science (B.C.S.)', 'Integrated B.Tech',
       'Master of Science (M.Sc) (Hons.)', 'Integrated M.Te

In [17]:
s1 = pd.Series(list(array_degree_code), name='degree_code')
s2 = pd.Series(list(array_degree), name='degree')
df_unique_combinations=pd.concat([s1, s2], axis=1)

In [18]:
df_unique_combinations

,degree_code,degree
0,11,Bachelor of Science (B.Sc)
1,12,Bachelor of Technology (B.Tech)
2,22,Master of Science (M.Sc)
3,9,Bachelor of Engineering (B.E)
4,31,NaN
5,25,PG Diploma in Data Science
6,30,Post Graduate Programme (PGP)
7,26,Post Graduate Diploma
8,20,Master of Computer Applications (MCA)
9,24,Master of Technology (M.Tech)


# Need to figure out which codes to combine to reduce the cardinality

In [19]:
#one set 
set1_codes=[11,12,9,1,5,10,7,4,6,8,2,3,0]#bachelor_programs
set2_codes=[22,20,24,17,21,15,23,18,14,16]#master_progrmas&integrated
set3_codes=[25,30,26,13,19,27,28,29]#post_graduate_mba program

In [20]:
len(set1_codes)+len(set2_codes)+len(set3_codes)

31

# combine codes

# update rows for bachelor_programs

In [21]:
# for each row,check whether degree_code is in set1_codes
list_flags=[False for i in range(len(dataset["Degree_code"]))]
#print(len(list_flags))     

In [22]:
list_code=list(dataset["Degree_code"].values)
#print(len(list_code))

In [23]:
list_index=list(range(len(dataset["Degree_code"].values)))
#print(len(list_index))

In [24]:
for (code,index) in zip(list_code,list_index):
    #print(code,index)
    if(code in set1_codes ):
        list_flags[index]=True 
print("no of rows belonging to set1 is :",sum(list_flags))

no of rows belonging to set1 is : 243


# check 1 corresponding row and check degree

In [25]:
len(list_flags)

392

In [26]:
len(dataset)

392

In [27]:
series_flags=pd.Series(list_flags, name='flags')
series_flags

0       True
1       True
2      False
3       True
4       True
       ...  
387     True
388    False
389     True
390     True
391    False
Name: flags, Length: 392, dtype: bool

# getting subset dataframe that belongs to set1-bachelor programs

In [28]:
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
0,ML0001,Bangalore,0,2,0,2,0,2,3,2,...,0,R Programming,Bachelor of Science (B.Sc),Mathematics,2017,NaN,NaN,NaN,NaN,11
1,ML0002,Bangalore,2,0,0,2,2,2,2,2,...,0,"Data Science, Machine Learning, Neural Network...",Bachelor of Technology (B.Tech),Computer Science & Engineering,2019,NaN,85.50/100,92.20/92.20,96.60/96.60,12
3,ML0004,Bangalore,2,0,2,1,0,2,0,0,...,0,"CSS, Deep Learning, Embedded Systems, HTML, Ma...",Bachelor of Engineering (B.E),Electronics and Communication,2019,NaN,6.86/10,76.00/76.00,84.32/84.32,9
4,ML0005,Bangalore,2,0,0,2,0,2,1,1,...,2,"HTML, OpenCV, Python, SQL, C++ Programming, CS...",Bachelor of Technology (B.Tech),Production Engineering,2018,NaN,6.38/10,65.20/65.20,68.80/68.80,12
10,ML0011,Bangalore,2,0,0,0,0,0,0,0,...,0,"CSS, HTML, Java, Python, SQL",Bachelor of Technology (B.Tech),Electronics and Communication,2018,NaN,64.00/100,95.00/95.00,NaN,12


In [29]:
dataset.loc[series_flags,'Degree_code']='bachelors program'

In [30]:
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
0,ML0001,Bangalore,0,2,0,2,0,2,3,2,...,0,R Programming,Bachelor of Science (B.Sc),Mathematics,2017,NaN,NaN,NaN,NaN,bachelors program
1,ML0002,Bangalore,2,0,0,2,2,2,2,2,...,0,"Data Science, Machine Learning, Neural Network...",Bachelor of Technology (B.Tech),Computer Science & Engineering,2019,NaN,85.50/100,92.20/92.20,96.60/96.60,bachelors program
3,ML0004,Bangalore,2,0,2,1,0,2,0,0,...,0,"CSS, Deep Learning, Embedded Systems, HTML, Ma...",Bachelor of Engineering (B.E),Electronics and Communication,2019,NaN,6.86/10,76.00/76.00,84.32/84.32,bachelors program
4,ML0005,Bangalore,2,0,0,2,0,2,1,1,...,2,"HTML, OpenCV, Python, SQL, C++ Programming, CS...",Bachelor of Technology (B.Tech),Production Engineering,2018,NaN,6.38/10,65.20/65.20,68.80/68.80,bachelors program
10,ML0011,Bangalore,2,0,0,0,0,0,0,0,...,0,"CSS, HTML, Java, Python, SQL",Bachelor of Technology (B.Tech),Electronics and Communication,2018,NaN,64.00/100,95.00/95.00,NaN,bachelors program


# update rows for master_integrated programs


In [31]:
list_flags=[False for i in range(len(dataset["Degree_code"]))]
list_code=list(dataset["Degree_code"].values)
list_index=list(range(len(dataset["Degree_code"].values)))
for (code,index) in zip(list_code,list_index):
    #print(code,index)
    if(code in set2_codes ):
        list_flags[index]=True 
print("no of rows belonging to set2 is :",sum(list_flags))

no of rows belonging to set2 is : 116


In [32]:
series_flags=pd.Series(list_flags, name='flags')

# getting subset dataframe that belongs to set2-master_integrated programs

In [33]:
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
2,ML0003,Bangalore,3,0,1,2,2,2,0,2,...,0,"Algorithms, Data Structures, Python, C Program...",Master of Science (M.Sc),Computer Science,2018,7.91/10,70.00/100,64.83/64.83,77.60/77.60,22
16,ML0017,Kozhikode,2,0,0,2,0,2,2,0,...,0,"MS-Office, Python, Data Science, Machine Learn...",Master of Science (M.Sc),Mathematics,2019,79.00/100,81.25/100,NaN,NaN,22
20,ML0021,Thrissur,1,0,0,2,2,3,3,2,...,0,"HTML, MS-Excel, SQL, Android, JavaScript, PHP,...",Master of Computer Applications (MCA),Computer Application,2017,NaN,NaN,NaN,NaN,20
21,ML0022,Bangalore,1,1,0,0,0,0,0,0,...,0,"Python, R Programming",Master of Science (M.Sc),Statistics,2019,3.76/4,3.56/4,80.50/80.50,85.85/85.85,22
24,ML0025,Lucknow,2,0,0,0,2,0,2,2,...,0,"Algorithms, C++ Programming, Data Structures, ...",Master of Computer Applications (MCA),NaN,2020,9.00/10,6.00/10,NaN,NaN,20


In [34]:
dataset.loc[series_flags,'Degree_code']='master_integrated program'
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
2,ML0003,Bangalore,3,0,1,2,2,2,0,2,...,0,"Algorithms, Data Structures, Python, C Program...",Master of Science (M.Sc),Computer Science,2018,7.91/10,70.00/100,64.83/64.83,77.60/77.60,master_integrated program
16,ML0017,Kozhikode,2,0,0,2,0,2,2,0,...,0,"MS-Office, Python, Data Science, Machine Learn...",Master of Science (M.Sc),Mathematics,2019,79.00/100,81.25/100,NaN,NaN,master_integrated program
20,ML0021,Thrissur,1,0,0,2,2,3,3,2,...,0,"HTML, MS-Excel, SQL, Android, JavaScript, PHP,...",Master of Computer Applications (MCA),Computer Application,2017,NaN,NaN,NaN,NaN,master_integrated program
21,ML0022,Bangalore,1,1,0,0,0,0,0,0,...,0,"Python, R Programming",Master of Science (M.Sc),Statistics,2019,3.76/4,3.56/4,80.50/80.50,85.85/85.85,master_integrated program
24,ML0025,Lucknow,2,0,0,0,2,0,2,2,...,0,"Algorithms, C++ Programming, Data Structures, ...",Master of Computer Applications (MCA),NaN,2020,9.00/10,6.00/10,NaN,NaN,master_integrated program


# update rows for post_graduate_mba program

In [35]:
list_flags=[False for i in range(len(dataset["Degree_code"]))]
list_code=list(dataset["Degree_code"].values)
list_index=list(range(len(dataset["Degree_code"].values)))
for (code,index) in zip(list_code,list_index):
    #print(code,index)
    if(code in set3_codes ):
        list_flags[index]=True 
print("no of rows belonging to set3 is :",sum(list_flags))

no of rows belonging to set3 is : 21


# getting subset dataframe that belongs to set3-post_graduate_mba programs

In [36]:
series_flags=pd.Series(list_flags, name='flags')
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
7,ML0008,Mumbai,3,1,0,0,0,2,0,0,...,0,"Data Analytics, Python, SQL, .NET, ASP.NET, C#...",PG Diploma in Data Science,NaN,2019,3.61/4,2.68/4,NaN,NaN,25
8,ML0009,Bangalore,3,0,0,0,0,2,2,0,...,0,"English Proficiency (Spoken), English Proficie...",Post Graduate Programme (PGP),Data Science,2020,8.00/10,7.00/10,70.00/70.00,93.00/93.00,30
9,ML0010,Bangalore,2,0,2,0,0,2,2,2,...,2,"MS-Office, Computer Vision, Data Analytics, De...",Post Graduate Programme (PGP),Artificial Intelligence And Machine Learning,2020,71.00/100,60.00/100,71.00/71.00,NaN,30
15,ML0016,Bangalore,2,0,0,0,2,2,2,1,...,0,"SQL, Data Analytics, MS-Excel, Machine Learnin...",Post Graduate Programme (PGP),Data Science And Engineering,2020,NaN,8.00/10,NaN,NaN,30
19,ML0020,Chennai,2,0,0,2,2,2,2,2,...,1,"BIG DATA ANALYTICS, Data Analytics, Database M...",Post Graduate Diploma,Data Science,2019,5.60/10,65.00/100,NaN,89.00/89.00,26


In [37]:
dataset.loc[series_flags,'Degree_code']='post_graduate_mba program'
dataset.loc[series_flags].head(5)

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
7,ML0008,Mumbai,3,1,0,0,0,2,0,0,...,0,"Data Analytics, Python, SQL, .NET, ASP.NET, C#...",PG Diploma in Data Science,NaN,2019,3.61/4,2.68/4,NaN,NaN,post_graduate_mba program
8,ML0009,Bangalore,3,0,0,0,0,2,2,0,...,0,"English Proficiency (Spoken), English Proficie...",Post Graduate Programme (PGP),Data Science,2020,8.00/10,7.00/10,70.00/70.00,93.00/93.00,post_graduate_mba program
9,ML0010,Bangalore,2,0,2,0,0,2,2,2,...,2,"MS-Office, Computer Vision, Data Analytics, De...",Post Graduate Programme (PGP),Artificial Intelligence And Machine Learning,2020,71.00/100,60.00/100,71.00/71.00,NaN,post_graduate_mba program
15,ML0016,Bangalore,2,0,0,0,2,2,2,1,...,0,"SQL, Data Analytics, MS-Excel, Machine Learnin...",Post Graduate Programme (PGP),Data Science And Engineering,2020,NaN,8.00/10,NaN,NaN,post_graduate_mba program
19,ML0020,Chennai,2,0,0,2,2,2,2,2,...,1,"BIG DATA ANALYTICS, Data Analytics, Database M...",Post Graduate Diploma,Data Science,2019,5.60/10,65.00/100,NaN,89.00/89.00,post_graduate_mba program


In [38]:
dataset['Degree_code'].describe()

count                   392
unique                    4
top       bachelors program
freq                    243
Name: Degree_code, dtype: object

In [39]:
dataset.groupby(['Degree_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Node.js (out of 3),ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
Degree_code,,,,,,,,,,,,,,,,,,,,,
31,12,12,12,12,12,12,12,12,12,12,...,12,12,12,0,12,12,3,12,6,6
bachelors program,243,243,243,243,243,243,243,243,243,243,...,243,243,232,243,234,243,4,211,147,137
master_integrated program,116,116,116,116,116,116,116,116,116,116,...,116,116,113,116,108,116,84,97,65,62
post_graduate_mba program,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,20,21,12,18,11,10


# combine both master_integrated program and post_graduate_mba program into one category,since they are very similar

In [40]:
filt=(dataset['Degree_code']=='post_graduate_mba program')
dataset.loc[filt,'Degree_code'] = 'post_graduate_program'


In [41]:
dataset.groupby(['Degree_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Node.js (out of 3),ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
Degree_code,,,,,,,,,,,,,,,,,,,,,
31,12,12,12,12,12,12,12,12,12,12,...,12,12,12,0,12,12,3,12,6,6
bachelors program,243,243,243,243,243,243,243,243,243,243,...,243,243,232,243,234,243,4,211,147,137
master_integrated program,116,116,116,116,116,116,116,116,116,116,...,116,116,113,116,108,116,84,97,65,62
post_graduate_program,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,20,21,12,18,11,10


In [42]:
filt=(dataset['Degree_code']=='master_integrated program')
dataset.loc[filt,'Degree_code'] = 'post_graduate_program'
dataset.groupby(['Degree_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Node.js (out of 3),ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
Degree_code,,,,,,,,,,,,,,,,,,,,,
31,12,12,12,12,12,12,12,12,12,12,...,12,12,12,0,12,12,3,12,6,6
bachelors program,243,243,243,243,243,243,243,243,243,243,...,243,243,232,243,234,243,4,211,147,137
post_graduate_program,137,137,137,137,137,137,137,137,137,137,...,137,137,134,137,128,137,96,115,76,72


# mode imputation for degree_code=31 ,representing NaN values.

In [43]:
filt_na_flag=dataset['Degree'].isna()

In [44]:
type(dataset['Degree'].isna())

pandas.core.series.Series

In [45]:
dataset.loc[filt_na_flag,['Degree','Degree_code']].head(10)

,Degree,Degree_code
5,NaN,31
6,NaN,31
12,NaN,31
52,NaN,31
64,NaN,31
101,NaN,31
117,NaN,31
137,NaN,31
192,NaN,31
212,NaN,31


In [46]:
st.mode(dataset.Degree_code)

'bachelors program'

In [47]:
dataset.loc[filt_na_flag,['Degree_code']]=st.mode(dataset.Degree_code)

In [48]:
dataset.groupby(['Degree_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Node.js (out of 3),ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
Degree_code,,,,,,,,,,,,,,,,,,,,,
bachelors program,255,255,255,255,255,255,255,255,255,255,...,255,255,244,243,246,255,7,223,153,143
post_graduate_program,137,137,137,137,137,137,137,137,137,137,...,137,137,134,137,128,137,96,115,76,72


In [49]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 24 columns):
Application_ID                392 non-null object
Current City                  392 non-null object
Python (out of 3)             392 non-null int64
R Programming (out of 3)      392 non-null int64
Deep Learning (out of 3)      392 non-null int64
PHP (out of 3)                392 non-null int64
MySQL (out of 3)              392 non-null int64
HTML (out of 3)               392 non-null int64
CSS (out of 3)                392 non-null int64
JavaScript (out of 3)         392 non-null int64
AJAX (out of 3)               392 non-null int64
Bootstrap (out of 3)          392 non-null int64
MongoDB (out of 3)            392 non-null int64
Node.js (out of 3)            392 non-null int64
ReactJS (out of 3)            392 non-null int64
Other skills                  378 non-null object
Degree                        380 non-null object
Stream                        374 non-null object
Current 

# See unique values for Stream and do similar things to what had been done to Degree

In [50]:
dataset.Stream.unique()

array(['Mathematics', 'Computer Science & Engineering',
       'Computer  Science', 'Electronics and Communication',
       'Production Engineering', 'Data Science Engineering',
       'Artificial Intelligence', nan, 'Data Science',
       'Artificial Intelligence And Machine Learning',
       'Advanced Pg Diploma In Artificial Intelligence',
       'Data Science And Engineering', 'Computer Application',
       'Statistics', 'Electronics and Instrumentation',
       'Digital Communication & Networking',
       'Electrical and Electronics Engineering', 'Business Analytics',
       'Mechanical Engineering', 'Computer Technology',
       'Information Systems', 'Information Technology',
       'CSE With Cloud Computing', 'Data Analytics', 'Science',
       'Biological sciences and bioengineering', 'Big Data Analytics',
       'Biotechnology', 'Mechatronics', 'computer science',
       'Instrumentation & Electronics', 'Civil Engineering',
       'Industrial and Management Engineering',
    

# there are too many values .high cardinality.need to club some of the values=>advantageous_set,normal_set
# advantageous set= anything that conatins math,analytics,stat,computer,data science,artificial intelligence,software,information
# normal_set=others

In [51]:
dataset["Stream_code"] = lb_Degree.fit_transform(list(dataset["Stream"]))
dataset[["Stream", "Stream_code"]].head(11)

,Stream,Stream_code
0,Mathematics,62
1,Computer Science & Engineering,22
2,Computer Science,17
3,Electronics and Communication,42
4,Production Engineering,67
5,Data Science Engineering,33
6,Artificial Intelligence,4
7,NaN,78
8,Data Science,31
9,Artificial Intelligence And Machine Learning,5


In [52]:
array_Stream_code=dataset.Stream_code.unique()
array_Stream=dataset.Stream.unique()
s1 = pd.Series(list(array_Stream_code), name='Stream_code')
s2 = pd.Series(list(array_Stream), name='Stream')
df_unique_combinations=pd.concat([s1, s2], axis=1)
df_unique_combinations

,Stream_code,Stream
0,62,Mathematics
1,22,Computer Science & Engineering
2,17,Computer Science
3,42,Electronics and Communication
4,67,Production Engineering
...,...,...
74,53,Information Science & Engineering
75,72,Software Engineering
76,21,Computer Network And Engg
77,71,Software Systems


# There is some data available as kannur university,which is irrelevant data with respect to stream.we have to treat it as null and impute with mode after binning.

# checking for university in stream

In [53]:
dataset["Stream"].str.contains("University").sum()

1

# only 1 wrong data.that will  be handled.Stream_code 58 treated as null.

# Need to figure out which codes to combine to reduce the cardinality

# Set1=advantageous,get row index of df_unique_combinations that have rows with stream name pattern

# first mark Stream as not defined for those na values

In [54]:
dataset["Stream"].isna().sum()

18

In [55]:
filt=dataset["Stream"].isna()

In [56]:
dataset.loc[filt,['Stream']]='Not defined'

In [57]:
dataset.loc[filt,['Stream']]

,Stream
7,Not defined
24,Not defined
68,Not defined
72,Not defined
99,Not defined
107,Not defined
128,Not defined
151,Not defined
204,Not defined
210,Not defined


In [58]:
filt=dataset["Stream"].str.contains("omputer")

In [59]:
list_omputer=dataset.index[filt].tolist()


In [60]:
adv_list=[]
adv_list.extend(list_omputer)

# 167 computer related streams

In [61]:
len(adv_list)

167

In [62]:
filt=dataset["Stream"].str.contains("Stat")
list_stat=dataset.index[filt].tolist()
adv_list.extend(list_stat)
len(adv_list)

170

In [63]:
filt=dataset["Stream"].str.contains("Data")
list_data=dataset.index[filt].tolist()
adv_list.extend(list_data)
len(adv_list)

193

In [64]:
filt=dataset["Stream"].str.contains("rtificial")
list_rtificial=dataset.index[filt].tolist()
adv_list.extend(list_rtificial)
len(adv_list)

200

In [65]:
filt=dataset["Stream"].str.contains("oftware")
list_oftware=dataset.index[filt].tolist()
adv_list.extend(list_oftware)
len(adv_list)

203

In [66]:
filt=dataset["Stream"].str.contains("nformation")
list_nformation=dataset.index[filt].tolist()
adv_list.extend(list_nformation)
len(adv_list)

231

In [67]:
filt=dataset["Stream"].str.contains("nalytic")
list_nalytic=dataset.index[filt].tolist()
adv_list.extend(list_nalytic)
len(adv_list)

247

In [68]:
filt=dataset["Stream"].str.contains("Math")
list_MATH=dataset.index[filt].tolist()
adv_list.extend(list_MATH)
len(adv_list)

255

# perform sanity check

In [69]:
dataset.loc[adv_list,'Stream'].head(5)

1     Computer Science & Engineering
2                  Computer  Science
13                 Computer  Science
14    Computer Science & Engineering
18    Computer Science & Engineering
Name: Stream, dtype: object

# check whether null values are there and assign stream code to 58

In [70]:
filt=dataset["Stream"].isna()
dataset.loc[filt,['Stream_code']]='58'

In [71]:
dataset.groupby(['Stream_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
Stream_code,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,0,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,0,1,1,1,1
3,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,0,2,2,0,2
4,3,3,3,3,3,3,3,3,3,3,...,3,3,1,3,3,0,3,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,2,2,2,3
75,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,0,0,1
76,11,11,11,11,11,11,11,11,11,11,...,11,9,11,11,11,4,7,4,2,11


# assign to Stream_code  to 2 if belonging to advantageous list,else 1

In [72]:
dataset.loc[adv_list,'Stream']

1        Computer Science & Engineering
2                     Computer  Science
13                    Computer  Science
14       Computer Science & Engineering
18       Computer Science & Engineering
                     ...               
111                         Mathematics
121                 Commerce With Maths
149                         Mathematics
213                         Mathematics
218    Computer Science And Mathematics
Name: Stream, Length: 255, dtype: object

In [73]:
dataset.loc[adv_list,'Stream_code']=2

In [74]:
filt= (dataset.Stream_code != 2) & (dataset.Stream_code !=58)

In [75]:
dataset.loc[filt,'Stream_code']

3      42
4      67
7      78
10     42
11     42
       ..
379    65
380    42
382    50
385    78
389    45
Name: Stream_code, Length: 148, dtype: int64

In [76]:
dataset.loc[filt,'Stream_code']=1

# impute 'not defined' rows with mode

In [77]:
filt=(dataset.Stream_code ==58)

In [78]:
st.mode(dataset.Stream_code)

2

In [79]:
dataset.loc[filt,['Stream_code']]=st.mode(dataset.Stream_code)

In [80]:
dataset.groupby(['Stream_code']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code
Stream_code,,,,,,,,,,,,,,,,,,,,,
1,148,148,148,148,148,148,148,148,148,148,...,148,139,146,148,148,23,129,88,85,148
2,244,244,244,244,244,244,244,244,244,244,...,244,239,234,244,244,80,209,141,130,244


# checking_data once again

In [81]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 25 columns):
Application_ID                392 non-null object
Current City                  392 non-null object
Python (out of 3)             392 non-null int64
R Programming (out of 3)      392 non-null int64
Deep Learning (out of 3)      392 non-null int64
PHP (out of 3)                392 non-null int64
MySQL (out of 3)              392 non-null int64
HTML (out of 3)               392 non-null int64
CSS (out of 3)                392 non-null int64
JavaScript (out of 3)         392 non-null int64
AJAX (out of 3)               392 non-null int64
Bootstrap (out of 3)          392 non-null int64
MongoDB (out of 3)            392 non-null int64
Node.js (out of 3)            392 non-null int64
ReactJS (out of 3)            392 non-null int64
Other skills                  378 non-null object
Degree                        380 non-null object
Stream                        392 non-null object
Current 

# need to work on other skills column

# identify na values

In [82]:
dataset['Other skills'].isna().sum()

14

# set na to not defined

In [83]:
filt=dataset['Other skills'].isna()
dataset.loc[filt,['Other skills']]='Not defined'

In [84]:
l=dataset['Other skills'].tolist()

In [85]:
import re
help(re.split)

Help on function split in module re:

split(pattern, string, maxsplit=0, flags=0)
    Split the source string by the occurrences of the pattern,
    returning a list containing the resulting substrings.  If
    capturing parentheses are used in pattern, then the text of all
    groups in the pattern are also returned as part of the resulting
    list.  If maxsplit is nonzero, at most maxsplit splits occur,
    and the remainder of the string is returned as the final element
    of the list.



# get list of all skills in a list

In [86]:
flat_list = []
#print(type(l))
for sublist in l:
    #print(sublist)
    #print(type(sublist))
    item_list=re.split(',',sublist)
    #print(item_list)
    flat_list.extend(item_list)
    #for item in sublist:
        #flat_list.append(item)
#print(flat_list)

In [87]:
def rem_space(skill):
    return skill.strip()

# remove leading and trailing spaces from all elements of the list

In [88]:
flat_list_Series = pd.Series(flat_list) 
flat_list_Series=flat_list_Series.apply(rem_space)

In [89]:
total_skills=len(flat_list_Series.unique())

In [90]:
skills_Series=dataset['Other skills'].str.split(",")

In [91]:
type(skills_Series[0])

list

In [92]:
len(skills_Series[0])

1

In [93]:
dataset['relative_count_skills']=skills_Series.apply(len)/total_skills

In [94]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 26 columns):
Application_ID                392 non-null object
Current City                  392 non-null object
Python (out of 3)             392 non-null int64
R Programming (out of 3)      392 non-null int64
Deep Learning (out of 3)      392 non-null int64
PHP (out of 3)                392 non-null int64
MySQL (out of 3)              392 non-null int64
HTML (out of 3)               392 non-null int64
CSS (out of 3)                392 non-null int64
JavaScript (out of 3)         392 non-null int64
AJAX (out of 3)               392 non-null int64
Bootstrap (out of 3)          392 non-null int64
MongoDB (out of 3)            392 non-null int64
Node.js (out of 3)            392 non-null int64
ReactJS (out of 3)            392 non-null int64
Other skills                  392 non-null object
Degree                        380 non-null object
Stream                        392 non-null object
Current 

# now lets come to current year of graduation.
# assume years of working as current year-year of graduation

In [95]:
now = datetime.datetime.now()
running_year=int(now.year)
print(running_year)

2020


In [96]:
dataset['working_years']=dataset['Current Year Of Graduation']-running_year

# reset working years to 0 ,if gone in negative,that means the candiddate has not yet graduated.

In [97]:
filt=(dataset['working_years']<0)
dataset.loc[filt,['working_years']] = 0

In [98]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 27 columns):
Application_ID                392 non-null object
Current City                  392 non-null object
Python (out of 3)             392 non-null int64
R Programming (out of 3)      392 non-null int64
Deep Learning (out of 3)      392 non-null int64
PHP (out of 3)                392 non-null int64
MySQL (out of 3)              392 non-null int64
HTML (out of 3)               392 non-null int64
CSS (out of 3)                392 non-null int64
JavaScript (out of 3)         392 non-null int64
AJAX (out of 3)               392 non-null int64
Bootstrap (out of 3)          392 non-null int64
MongoDB (out of 3)            392 non-null int64
Node.js (out of 3)            392 non-null int64
ReactJS (out of 3)            392 non-null int64
Other skills                  392 non-null object
Degree                        380 non-null object
Stream                        392 non-null object
Current 

# Assuming current_city = bangalore as ideal since hiring company is assumed to be placed in bangalore and chennai.will bin all values other  than bangalore,chennai in one group and all values in bangalore,chennai to be another group.

In [99]:
list_city_binning=['Other' for i in range(len(dataset["Current City"]))]
print(len(list_city_binning))

392


In [100]:
dataset['City_binned']=pd.Series(list_city_binning, name='City_binned')

In [101]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 28 columns):
Application_ID                392 non-null object
Current City                  392 non-null object
Python (out of 3)             392 non-null int64
R Programming (out of 3)      392 non-null int64
Deep Learning (out of 3)      392 non-null int64
PHP (out of 3)                392 non-null int64
MySQL (out of 3)              392 non-null int64
HTML (out of 3)               392 non-null int64
CSS (out of 3)                392 non-null int64
JavaScript (out of 3)         392 non-null int64
AJAX (out of 3)               392 non-null int64
Bootstrap (out of 3)          392 non-null int64
MongoDB (out of 3)            392 non-null int64
Node.js (out of 3)            392 non-null int64
ReactJS (out of 3)            392 non-null int64
Other skills                  392 non-null object
Degree                        380 non-null object
Stream                        392 non-null object
Current 

In [102]:
filt=(dataset['Current City']=='Bangalore')|(dataset['Current City']=='Chennai')

In [103]:
dataset.loc[filt,['City_binned']]='Advantage'

In [104]:
dataset.groupby(['City_binned']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code,Stream_code,relative_count_skills,working_years
City_binned,,,,,,,,,,,,,,,,,,,,,
Advantage,226,226,226,226,226,226,226,226,226,226,...,226,226,74,189,120,114,226,226,226,226
Other,166,166,166,166,166,166,166,166,166,166,...,166,166,29,149,109,101,166,166,166,166


# convert into numerical considering bangalore,chennai have double advantage over others.

In [105]:
filt=(dataset['City_binned']=='Advantage')
dataset.loc[filt,['City_binned']]=2

In [106]:
filt=(dataset['City_binned']=='Other')
dataset.loc[filt,['City_binned']]=1

In [107]:
dataset.describe()

,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),AJAX (out of 3),Bootstrap (out of 3),MongoDB (out of 3),Node.js (out of 3),ReactJS (out of 3),Current Year Of Graduation,Stream_code,relative_count_skills,working_years,City_binned
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,1.375000,0.566327,0.461735,0.612245,0.403061,1.346939,1.045918,0.770408,0.015306,0.265306,0.035714,0.086735,0.178571,2018.867347,1.622449,0.047656,0.160714,1.576531
std,0.975237,0.905052,0.842336,0.911789,0.837602,1.071386,1.022976,0.966626,0.122924,0.715928,0.255377,0.401567,0.499360,2.280296,0.485394,0.031822,0.471367,0.494740
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2004.000000,1.000000,0.006250,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2018.750000,1.000000,0.031250,0.000000,1.000000
50%,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019.000000,2.000000,0.043750,0.000000,2.000000
75%,2.000000,1.000000,1.000000,1.000000,0.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2020.000000,2.000000,0.062500,0.000000,2.000000
max,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,3.000000,2.000000,2.000000,2.000000,2023.000000,2.000000,0.250000,3.000000,2.000000


In [108]:
dataset.groupby(['City_binned']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Degree_code,Stream_code,relative_count_skills,working_years
City_binned,,,,,,,,,,,,,,,,,,,,,
1,166,166,166,166,166,166,166,166,166,166,...,166,166,29,149,109,101,166,166,166,166
2,226,226,226,226,226,226,226,226,226,226,...,226,226,74,189,120,114,226,226,226,226


# change performance columns to floats

In [109]:
dataset['Performance_PG'].isna().sum()

289

# -1 to mark na values

In [110]:
filt=dataset['Performance_PG'].isna()
dataset.loc[filt,'Performance_PG']='-1/1'
filt=dataset['Performance_UG'].isna()
dataset.loc[filt,'Performance_UG']='-1/1'
filt=dataset['Performance_10'].isna()
dataset.loc[filt,'Performance_10']='-100/100'
filt=dataset['Performance_12'].isna()
dataset.loc[filt,'Performance_12']='-100/100'

# split string and convert into two values 

In [111]:
l=dataset['Performance_PG'].tolist()
flat_list=[]
#print(type(l))
for sublist in l:
    #print(sublist)
    #print(type(sublist))
    item_list=re.split('/',sublist)
    #print(item_list)
    flat_list.append(item_list)

In [112]:
pg_perf_subset=pd.DataFrame(flat_list)
pg_perf_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 2 columns):
0    392 non-null object
1    392 non-null object
dtypes: object(2)
memory usage: 6.2+ KB


# convert string to float

In [113]:
pg_perf_subset.columns = ['performance_pg_ur','performance_pg_tot']
pg_perf_subset['performance_pg_ur'] =  pg_perf_subset['performance_pg_ur'].astype(float)
pg_perf_subset['performance_pg_tot'] =  pg_perf_subset['performance_pg_tot'].astype(float)
print(pg_perf_subset['performance_pg_ur'].head(5))
print(pg_perf_subset['performance_pg_tot'].head(5))

0   -1.00
1   -1.00
2    7.91
3   -1.00
4   -1.00
Name: performance_pg_ur, dtype: float64
0     1.0
1     1.0
2    10.0
3     1.0
4     1.0
Name: performance_pg_tot, dtype: float64


In [114]:
pg_perf_subset['performance_pg_percentage']=pg_perf_subset['performance_pg_ur']/pg_perf_subset['performance_pg_tot']

In [115]:
l=dataset['Performance_UG'].tolist()
flat_list=[]
#print(type(l))
for sublist in l:
    #print(sublist)
    #print(type(sublist))
    item_list=re.split('/',sublist)
    #print(item_list)
    flat_list.append(item_list)


ug_perf_subset=pd.DataFrame(flat_list)
ug_perf_subset.info()

ug_perf_subset.columns = ['performance_ug_ur','performance_ug_tot']
ug_perf_subset['performance_ug_ur'] =  ug_perf_subset['performance_ug_ur'].astype(float)
ug_perf_subset['performance_ug_tot'] =  ug_perf_subset['performance_ug_tot'].astype(float)
ug_perf_subset['performance_ug_ur'].head(5)
ug_perf_subset['performance_ug_tot'].head(5)
ug_perf_subset['performance_ug_percentage']=ug_perf_subset['performance_ug_ur']/ug_perf_subset['performance_ug_tot']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 2 columns):
0    392 non-null object
1    392 non-null object
dtypes: object(2)
memory usage: 6.2+ KB


In [116]:
l=dataset['Performance_10'].tolist()
flat_list=[]
#print(type(l))
for sublist in l:
    #print(sublist)
    #print(type(sublist))
    item_list=re.split('/',sublist)
    #print(item_list)
    flat_list.append(item_list)


std10_perf_subset=pd.DataFrame(flat_list)
std10_perf_subset.info()

std10_perf_subset.columns = ['performance_std10_ur','performance_std10_tot']
std10_perf_subset['performance_std10_ur'] =  std10_perf_subset['performance_std10_ur'].astype(float)
std10_perf_subset['performance_std10_tot'] =  100
std10_perf_subset['performance_std10_ur'].head(5)
std10_perf_subset['performance_std10_tot'].head(5)
std10_perf_subset['performance_std10_percentage']=std10_perf_subset['performance_std10_ur']/std10_perf_subset['performance_std10_tot']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 2 columns):
0    392 non-null object
1    392 non-null object
dtypes: object(2)
memory usage: 6.2+ KB


In [117]:
l=dataset['Performance_12'].tolist()
flat_list=[]
#print(type(l))
for sublist in l:
    #print(sublist)
    #print(type(sublist))
    item_list=re.split('/',sublist)
    #print(item_list)
    flat_list.append(item_list)


std12_perf_subset=pd.DataFrame(flat_list)
std12_perf_subset.info()

std12_perf_subset.columns = ['performance_std12_ur','performance_std12_tot']
std12_perf_subset['performance_std12_ur'] =  std12_perf_subset['performance_std12_ur'].astype(float)
std12_perf_subset['performance_std12_tot'] =  100
std12_perf_subset['performance_std12_ur'].head(5)
std12_perf_subset['performance_std12_tot'].head(5)
std12_perf_subset['performance_std12_percentage']=std12_perf_subset['performance_std12_ur']/std12_perf_subset['performance_std12_tot']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 2 columns):
0    392 non-null object
1    392 non-null object
dtypes: object(2)
memory usage: 6.2+ KB


#  -1 denotes nan values

In [118]:
pg_perf_subset.describe()

,performance_pg_ur,performance_pg_tot,performance_pg_percentage
count,392.000000,392.000000,392.000000
mean,6.083444,10.872449,-0.541817
std,18.779202,27.258712,0.771588
min,-1.000000,1.000000,-1.000000
25%,-1.000000,1.000000,-1.000000
50%,-1.000000,1.000000,-1.000000
75%,6.000000,7.000000,0.600000
max,80.000000,100.000000,1.000000


# merge back with the main dataset

In [119]:
dataset['performance_pg_percentage']=pg_perf_subset['performance_pg_percentage']
dataset['performance_ug_percentage']=ug_perf_subset['performance_ug_percentage']
dataset['performance_std10_percentage']=std10_perf_subset['performance_std10_percentage']
dataset['performance_std12_percentage']=std12_perf_subset['performance_std12_percentage']

In [120]:
std10_perf_subset['performance_std10_percentage'].unique()

array([-1.    ,  0.966 ,  0.776 ,  0.8432,  0.688 ,  0.8   ,  0.93  ,
        0.088 ,  0.726 ,  0.095 ,  0.972 ,  0.89  ,  0.8585,  0.1   ,
        0.76  ,  0.81  ,  0.7152,  0.085 ,  0.084 ,  0.8768,  0.87  ,
        0.092 ,  0.794 ,  0.962 ,  0.84  ,  0.662 ,  0.0742,  0.087 ,
        0.798 ,  0.7545,  0.815 ,  0.098 ,  0.7981,  0.08  ,  0.093 ,
        0.845 ,  0.75  ,  0.73  ,  0.876 ,  0.646 ,  0.67  ,  0.744 ,
        0.09  ,  0.7953,  0.078 ,  0.074 ,  0.8704,  0.82  ,  0.821 ,
        0.687 ,  0.082 ,  0.822 ,  0.8233,  0.8672,  0.64  ,  0.63  ,
        0.88  ,  0.956 ,  0.081 ,  0.8982,  0.6816,  0.864 ,  0.7   ,
        0.868 ,  0.062 ,  0.094 ,  0.912 ,  0.8561,  0.952 ,  0.675 ,
        0.86  ,  0.92  ,  0.933 ,  0.942 ,  0.65  ,  0.097 ,  0.7183,
        0.62  ,  0.072 ,  0.735 ,  0.076 ,  0.077 ,  0.702 ,  0.66  ,
        0.6   ,  0.94  ,  0.7444,  0.086 ,  0.7683,  0.7968,  0.85  ,
        0.0666,  0.066 ,  0.6956,  0.816 ,  0.68  ,  0.635 ,  0.07  ,
        0.765 ,  0.6

# mark back all -1 values to null so that imputation can be done

In [121]:
filt=(dataset['performance_pg_percentage']==-1)
dataset.loc[filt,['performance_pg_percentage']]=None
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 32 columns):
Application_ID                  392 non-null object
Current City                    392 non-null object
Python (out of 3)               392 non-null int64
R Programming (out of 3)        392 non-null int64
Deep Learning (out of 3)        392 non-null int64
PHP (out of 3)                  392 non-null int64
MySQL (out of 3)                392 non-null int64
HTML (out of 3)                 392 non-null int64
CSS (out of 3)                  392 non-null int64
JavaScript (out of 3)           392 non-null int64
AJAX (out of 3)                 392 non-null int64
Bootstrap (out of 3)            392 non-null int64
MongoDB (out of 3)              392 non-null int64
Node.js (out of 3)              392 non-null int64
ReactJS (out of 3)              392 non-null int64
Other skills                    392 non-null object
Degree                          380 non-null object
Stream                  

In [122]:
filt=(dataset['performance_ug_percentage']==-1)
dataset.loc[filt,['performance_ug_percentage']]=None
filt=(dataset['performance_std10_percentage']==-1)
dataset.loc[filt,['performance_std10_percentage']]=None
filt=(dataset['performance_std12_percentage']==-1)
dataset.loc[filt,['performance_std12_percentage']]=None

In [123]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 32 columns):
Application_ID                  392 non-null object
Current City                    392 non-null object
Python (out of 3)               392 non-null int64
R Programming (out of 3)        392 non-null int64
Deep Learning (out of 3)        392 non-null int64
PHP (out of 3)                  392 non-null int64
MySQL (out of 3)                392 non-null int64
HTML (out of 3)                 392 non-null int64
CSS (out of 3)                  392 non-null int64
JavaScript (out of 3)           392 non-null int64
AJAX (out of 3)                 392 non-null int64
Bootstrap (out of 3)            392 non-null int64
MongoDB (out of 3)              392 non-null int64
Node.js (out of 3)              392 non-null int64
ReactJS (out of 3)              392 non-null int64
Other skills                    392 non-null object
Degree                          380 non-null object
Stream                  

# now that all numerical values are in proper place,we can go for imputing the missing values of performance columns using model based imputation.

In [124]:
dataset.columns

Index(['Application_ID', 'Current City', 'Python (out of 3)',
       'R Programming (out of 3)', 'Deep Learning (out of 3)',
       'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)',
       'CSS (out of 3)', 'JavaScript (out of 3)', 'AJAX (out of 3)',
       'Bootstrap (out of 3)', 'MongoDB (out of 3)', 'Node.js (out of 3)',
       'ReactJS (out of 3)', 'Other skills', 'Degree', 'Stream',
       'Current Year Of Graduation', 'Performance_PG', 'Performance_UG',
       'Performance_12', 'Performance_10', 'Degree_code', 'Stream_code',
       'relative_count_skills', 'working_years', 'City_binned',
       'performance_pg_percentage', 'performance_ug_percentage',
       'performance_std10_percentage', 'performance_std12_percentage'],
      dtype='object')

In [125]:
subset_cols=['Python (out of 3)',
       'R Programming (out of 3)', 'Deep Learning (out of 3)',
       'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)',
       'CSS (out of 3)', 'JavaScript (out of 3)', 'AJAX (out of 3)',
       'Bootstrap (out of 3)', 'MongoDB (out of 3)', 'Node.js (out of 3)',
       'ReactJS (out of 3)',
       'performance_pg_percentage', 'performance_ug_percentage',
       'performance_std10_percentage', 'performance_std12_percentage','Stream_code','relative_count_skills', 'working_years','City_binned']

In [126]:
subset_dataset=dataset.loc[0:,['Python (out of 3)',
       'R Programming (out of 3)', 'Deep Learning (out of 3)',
       'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)',
       'CSS (out of 3)', 'JavaScript (out of 3)', 'AJAX (out of 3)',
       'Bootstrap (out of 3)', 'MongoDB (out of 3)', 'Node.js (out of 3)',
       'ReactJS (out of 3)',
       'performance_pg_percentage', 'performance_ug_percentage',
       'performance_std10_percentage', 'performance_std12_percentage','Stream_code','relative_count_skills', 'working_years','City_binned']]

In [127]:
dataset.performance_std10_percentage.unique()

array([   nan, 0.966 , 0.776 , 0.8432, 0.688 , 0.8   , 0.93  , 0.088 ,
       0.726 , 0.095 , 0.972 , 0.89  , 0.8585, 0.1   , 0.76  , 0.81  ,
       0.7152, 0.085 , 0.084 , 0.8768, 0.87  , 0.092 , 0.794 , 0.962 ,
       0.84  , 0.662 , 0.0742, 0.087 , 0.798 , 0.7545, 0.815 , 0.098 ,
       0.7981, 0.08  , 0.093 , 0.845 , 0.75  , 0.73  , 0.876 , 0.646 ,
       0.67  , 0.744 , 0.09  , 0.7953, 0.078 , 0.074 , 0.8704, 0.82  ,
       0.821 , 0.687 , 0.082 , 0.822 , 0.8233, 0.8672, 0.64  , 0.63  ,
       0.88  , 0.956 , 0.081 , 0.8982, 0.6816, 0.864 , 0.7   , 0.868 ,
       0.062 , 0.094 , 0.912 , 0.8561, 0.952 , 0.675 , 0.86  , 0.92  ,
       0.933 , 0.942 , 0.65  , 0.097 , 0.7183, 0.62  , 0.072 , 0.735 ,
       0.076 , 0.077 , 0.702 , 0.66  , 0.6   , 0.94  , 0.7444, 0.086 ,
       0.7683, 0.7968, 0.85  , 0.0666, 0.066 , 0.6956, 0.816 , 0.68  ,
       0.635 , 0.07  , 0.765 , 0.6071, 0.7671, 0.832 , 0.95  , 0.71  ,
       0.8192, 0.7833, 0.78  , 0.936 , 0.74  ])

In [128]:
imputer = KNNImputer(n_neighbors=2)

In [129]:
subset_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 21 columns):
Python (out of 3)               392 non-null int64
R Programming (out of 3)        392 non-null int64
Deep Learning (out of 3)        392 non-null int64
PHP (out of 3)                  392 non-null int64
MySQL (out of 3)                392 non-null int64
HTML (out of 3)                 392 non-null int64
CSS (out of 3)                  392 non-null int64
JavaScript (out of 3)           392 non-null int64
AJAX (out of 3)                 392 non-null int64
Bootstrap (out of 3)            392 non-null int64
MongoDB (out of 3)              392 non-null int64
Node.js (out of 3)              392 non-null int64
ReactJS (out of 3)              392 non-null int64
performance_pg_percentage       103 non-null float64
performance_ug_percentage       338 non-null float64
performance_std10_percentage    215 non-null float64
performance_std12_percentage    229 non-null float64
Stream_code         

In [130]:
subset_dataset_filled = imputer.fit_transform(subset_dataset)

In [131]:
subset_dataset.performance_std10_percentage.unique()

array([   nan, 0.966 , 0.776 , 0.8432, 0.688 , 0.8   , 0.93  , 0.088 ,
       0.726 , 0.095 , 0.972 , 0.89  , 0.8585, 0.1   , 0.76  , 0.81  ,
       0.7152, 0.085 , 0.084 , 0.8768, 0.87  , 0.092 , 0.794 , 0.962 ,
       0.84  , 0.662 , 0.0742, 0.087 , 0.798 , 0.7545, 0.815 , 0.098 ,
       0.7981, 0.08  , 0.093 , 0.845 , 0.75  , 0.73  , 0.876 , 0.646 ,
       0.67  , 0.744 , 0.09  , 0.7953, 0.078 , 0.074 , 0.8704, 0.82  ,
       0.821 , 0.687 , 0.082 , 0.822 , 0.8233, 0.8672, 0.64  , 0.63  ,
       0.88  , 0.956 , 0.081 , 0.8982, 0.6816, 0.864 , 0.7   , 0.868 ,
       0.062 , 0.094 , 0.912 , 0.8561, 0.952 , 0.675 , 0.86  , 0.92  ,
       0.933 , 0.942 , 0.65  , 0.097 , 0.7183, 0.62  , 0.072 , 0.735 ,
       0.076 , 0.077 , 0.702 , 0.66  , 0.6   , 0.94  , 0.7444, 0.086 ,
       0.7683, 0.7968, 0.85  , 0.0666, 0.066 , 0.6956, 0.816 , 0.68  ,
       0.635 , 0.07  , 0.765 , 0.6071, 0.7671, 0.832 , 0.95  , 0.71  ,
       0.8192, 0.7833, 0.78  , 0.936 , 0.74  ])

In [132]:
type(subset_dataset_filled)

numpy.ndarray

In [133]:
subset_dataset_filled

array([[0.     , 2.     , 0.     , ..., 0.00625, 0.     , 2.     ],
       [2.     , 0.     , 0.     , ..., 0.03125, 0.     , 2.     ],
       [3.     , 0.     , 1.     , ..., 0.10625, 0.     , 2.     ],
       ...,
       [1.     , 0.     , 0.     , ..., 0.025  , 0.     , 1.     ],
       [2.     , 2.     , 0.     , ..., 0.03125, 0.     , 1.     ],
       [2.     , 3.     , 0.     , ..., 0.05   , 0.     , 1.     ]])

# filling original dataset with imputed values

In [134]:
print(len(subset_dataset_filled))
print(subset_dataset_filled.shape)

392
(392, 21)


In [135]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 32 columns):
Application_ID                  392 non-null object
Current City                    392 non-null object
Python (out of 3)               392 non-null int64
R Programming (out of 3)        392 non-null int64
Deep Learning (out of 3)        392 non-null int64
PHP (out of 3)                  392 non-null int64
MySQL (out of 3)                392 non-null int64
HTML (out of 3)                 392 non-null int64
CSS (out of 3)                  392 non-null int64
JavaScript (out of 3)           392 non-null int64
AJAX (out of 3)                 392 non-null int64
Bootstrap (out of 3)            392 non-null int64
MongoDB (out of 3)              392 non-null int64
Node.js (out of 3)              392 non-null int64
ReactJS (out of 3)              392 non-null int64
Other skills                    392 non-null object
Degree                          380 non-null object
Stream                  

In [136]:
subset_dataset_filled[0:392,13]

array([0.6885    , 0.655     , 0.791     , 0.719     , 0.8115    ,
       0.778     , 0.50578571, 0.9025    , 0.8       , 0.71      ,
       0.665     , 0.704     , 0.708     , 0.874     , 0.665     ,
       0.61      , 0.79      , 0.719     , 0.77      , 0.56      ,
       0.76      , 0.94      , 0.719     , 0.855     , 0.9       ,
       0.835     , 0.71      , 0.767     , 0.705     , 0.36095   ,
       0.8       , 0.739     , 0.704     , 0.78      , 0.76      ,
       0.75      , 0.605     , 0.665     , 0.68      , 0.665     ,
       0.93428571, 0.7695    , 0.74      , 0.77      , 0.8205    ,
       0.855     , 0.653     , 0.719     , 0.778     , 0.699     ,
       0.4425    , 0.75      , 0.58      , 0.7115    , 0.39095   ,
       0.755     , 0.8025    , 0.784     , 0.75      , 0.7835    ,
       0.855     , 0.8725    , 0.61      , 0.77      , 0.734     ,
       0.7335    , 0.766     , 0.77      , 0.665     , 0.665     ,
       0.806     , 0.6       , 0.908     , 0.85714286, 0.84778

In [137]:
dataset['performance_pg_percentage']

0        NaN
1        NaN
2      0.791
3        NaN
4        NaN
       ...  
387      NaN
388    0.778
389      NaN
390      NaN
391    0.640
Name: performance_pg_percentage, Length: 392, dtype: float64

# 13,14,15,16 are numpy column indices,which need to be filled/propagated into original dataset.

In [138]:
dataset['performance_pg_percentage']=pd.Series(subset_dataset_filled[0:392,13])
dataset['performance_ug_percentage']=pd.Series(subset_dataset_filled[0:392,14])


In [139]:
subset_dataset_filled[0:392,15]

array([0.75165, 0.966  , 0.776  , 0.8432 , 0.688  , 0.796  , 0.8    ,
       0.7208 , 0.93   , 0.509  , 0.095  , 0.4686 , 0.088  , 0.726  ,
       0.095  , 0.46   , 0.089  , 0.972  , 0.6984 , 0.89   , 0.913  ,
       0.8585 , 0.1    , 0.8811 , 0.3976 , 0.76   , 0.86   , 0.8811 ,
       0.8765 , 0.7972 , 0.81   , 0.7152 , 0.806  , 0.085  , 0.93   ,
       0.084  , 0.8768 , 0.5115 , 0.87   , 0.092  , 0.794  , 0.962  ,
       0.84   , 0.662  , 0.1    , 0.0742 , 0.095  , 0.1    , 0.4625 ,
       0.81415, 0.087  , 0.68215, 0.4935 , 0.798  , 0.7545 , 0.815  ,
       0.79715, 0.098  , 0.3725 , 0.098  , 0.4991 , 0.098  , 0.7981 ,
       0.8496 , 0.74355, 0.08   , 0.093  , 0.8496 , 0.675  , 0.355  ,
       0.81605, 0.845  , 0.7812 , 0.449  , 0.75   , 0.08   , 0.1    ,
       0.485  , 0.73   , 0.876  , 0.08   , 0.73005, 0.098  , 0.355  ,
       0.7671 , 0.4285 , 0.646  , 0.091  , 0.67   , 0.0815 , 0.744  ,
       0.09   , 0.355  , 0.095  , 0.928  , 0.7953 , 0.67   , 0.8496 ,
       0.078  , 0.81

In [140]:
dataset['performance_std10_percentage']=pd.Series(subset_dataset_filled[0:392,15])
dataset['performance_std12_percentage']=pd.Series(subset_dataset_filled[0:392,16])

In [141]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 32 columns):
Application_ID                  392 non-null object
Current City                    392 non-null object
Python (out of 3)               392 non-null int64
R Programming (out of 3)        392 non-null int64
Deep Learning (out of 3)        392 non-null int64
PHP (out of 3)                  392 non-null int64
MySQL (out of 3)                392 non-null int64
HTML (out of 3)                 392 non-null int64
CSS (out of 3)                  392 non-null int64
JavaScript (out of 3)           392 non-null int64
AJAX (out of 3)                 392 non-null int64
Bootstrap (out of 3)            392 non-null int64
MongoDB (out of 3)              392 non-null int64
Node.js (out of 3)              392 non-null int64
ReactJS (out of 3)              392 non-null int64
Other skills                    392 non-null object
Degree                          380 non-null object
Stream                  

# subsetting dataset to get relevant numerical columns and also leave out irrlevant columns that dont contribute to any variation in ranking the candidates

In [142]:
dataset.columns

Index(['Application_ID', 'Current City', 'Python (out of 3)',
       'R Programming (out of 3)', 'Deep Learning (out of 3)',
       'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)',
       'CSS (out of 3)', 'JavaScript (out of 3)', 'AJAX (out of 3)',
       'Bootstrap (out of 3)', 'MongoDB (out of 3)', 'Node.js (out of 3)',
       'ReactJS (out of 3)', 'Other skills', 'Degree', 'Stream',
       'Current Year Of Graduation', 'Performance_PG', 'Performance_UG',
       'Performance_12', 'Performance_10', 'Degree_code', 'Stream_code',
       'relative_count_skills', 'working_years', 'City_binned',
       'performance_pg_percentage', 'performance_ug_percentage',
       'performance_std10_percentage', 'performance_std12_percentage'],
      dtype='object')

In [143]:
subset_cols=['City_binned','Python (out of 3)',
       'R Programming (out of 3)', 'Deep Learning (out of 3)',
       'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)',
       'CSS (out of 3)', 'JavaScript (out of 3)', 'AJAX (out of 3)',
       'Bootstrap (out of 3)', 'MongoDB (out of 3)', 'Node.js (out of 3)',
       'ReactJS (out of 3)','relative_count_skills','Degree_code', 'Stream_code','working_years', 'performance_pg_percentage', 'performance_ug_percentage',
       'performance_std10_percentage', 'performance_std12_percentage']

In [144]:
final_df=dataset.loc[0:,['City_binned','Python (out of 3)',
       'R Programming (out of 3)', 'Deep Learning (out of 3)',
       'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)',
       'CSS (out of 3)', 'JavaScript (out of 3)', 'AJAX (out of 3)',
       'Bootstrap (out of 3)', 'MongoDB (out of 3)', 'Node.js (out of 3)',
       'ReactJS (out of 3)','relative_count_skills','Degree_code', 'Stream_code','working_years', 'performance_pg_percentage', 'performance_ug_percentage',
       'performance_std10_percentage', 'performance_std12_percentage']]

In [145]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 22 columns):
City_binned                     392 non-null int64
Python (out of 3)               392 non-null int64
R Programming (out of 3)        392 non-null int64
Deep Learning (out of 3)        392 non-null int64
PHP (out of 3)                  392 non-null int64
MySQL (out of 3)                392 non-null int64
HTML (out of 3)                 392 non-null int64
CSS (out of 3)                  392 non-null int64
JavaScript (out of 3)           392 non-null int64
AJAX (out of 3)                 392 non-null int64
Bootstrap (out of 3)            392 non-null int64
MongoDB (out of 3)              392 non-null int64
Node.js (out of 3)              392 non-null int64
ReactJS (out of 3)              392 non-null int64
relative_count_skills           392 non-null float64
Degree_code                     392 non-null object
Stream_code                     392 non-null int64
working_years            

# converting degree_code to numerical


In [146]:
final_df['Degree_code'].unique()

array(['bachelors program', 'post_graduate_program'], dtype=object)

# higher_weightage to post_graduate_program

In [147]:
filt=(final_df['Degree_code']=='bachelors program')
final_df.loc[filt,['Degree_code']]=1
filt=(final_df['Degree_code']=='post_graduate_program')
final_df.loc[filt,['Degree_code']]=2

# need to perform min-max normalization for clustering data 

In [148]:
normalized_final_df=(final_df-final_df.min())/(final_df.max()-final_df.min())

# next is k-means clustering with k=2,one cluster -high performing,another is normal
# those rows that belong to high performing cluster_center will go through to next round ,process whateverfrom sklearn.cluster import KMeansfrom sklearn.cluster import KMeans

In [149]:
from sklearn.cluster import KMeans

In [150]:
clf=KMeans(n_clusters=2)

In [151]:
clf.fit(normalized_final_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [152]:
centroids=clf.cluster_centers_
labels=clf.labels_

In [153]:
len(labels)

392

In [154]:
type(labels)

numpy.ndarray

In [155]:
dataset['cluster_type']=pd.Series(labels)

# finding the high performance cluster

In [156]:
type(centroids)

numpy.ndarray

In [157]:
print(centroids)

[[ 6.01156069e-01  4.73988439e-01  1.81117534e-01  1.36801541e-01
   3.83429672e-01  2.69749518e-01  7.18689788e-01  6.53179191e-01
   4.54720617e-01  3.46820809e-02  1.88824663e-01  2.60115607e-02
   7.22543353e-02  1.01156069e-01  2.12390692e-01  5.20231214e-01
   8.20809249e-01  5.58766859e-02  7.03787688e-01  6.67959917e-01
   5.37920346e-01  6.95606936e-01]
 [ 5.57077626e-01  4.45966514e-01  1.94824962e-01  1.67427702e-01
   6.24048706e-02  2.73972603e-02  2.35920852e-01  1.08066971e-01
   1.00456621e-01 -4.68375339e-17  9.13242009e-03  1.14155251e-02
   2.05479452e-02  7.99086758e-02  1.36283808e-01  2.14611872e-01
   4.65753425e-01  5.17503805e-02  7.42535953e-01  6.92888419e-01
   5.46474986e-01  6.58440688e-01]]


# centroids=clf.cluster_centers_
# labels=clf.labels_
# Value of K was chosen as 2 since we want two categories-one high performing and another -low performing.
# Had to use value of centroid to determine which among the clusters was the high performing one.


# clearly cluster 1 is the high performing one.All the records belonging to cluster 1 are eligible for next round.

# histogram for numeric data

In [158]:
dataset.columns

Index(['Application_ID', 'Current City', 'Python (out of 3)',
       'R Programming (out of 3)', 'Deep Learning (out of 3)',
       'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)',
       'CSS (out of 3)', 'JavaScript (out of 3)', 'AJAX (out of 3)',
       'Bootstrap (out of 3)', 'MongoDB (out of 3)', 'Node.js (out of 3)',
       'ReactJS (out of 3)', 'Other skills', 'Degree', 'Stream',
       'Current Year Of Graduation', 'Performance_PG', 'Performance_UG',
       'Performance_12', 'Performance_10', 'Degree_code', 'Stream_code',
       'relative_count_skills', 'working_years', 'City_binned',
       'performance_pg_percentage', 'performance_ug_percentage',
       'performance_std10_percentage', 'performance_std12_percentage',
       'cluster_type'],
      dtype='object')

In [163]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 33 columns):
Application_ID                  392 non-null object
Current City                    392 non-null object
Python (out of 3)               392 non-null int64
R Programming (out of 3)        392 non-null int64
Deep Learning (out of 3)        392 non-null int64
PHP (out of 3)                  392 non-null int64
MySQL (out of 3)                392 non-null int64
HTML (out of 3)                 392 non-null int64
CSS (out of 3)                  392 non-null int64
JavaScript (out of 3)           392 non-null int64
AJAX (out of 3)                 392 non-null int64
Bootstrap (out of 3)            392 non-null int64
MongoDB (out of 3)              392 non-null int64
Node.js (out of 3)              392 non-null int64
ReactJS (out of 3)              392 non-null int64
Other skills                    392 non-null object
Degree                          380 non-null object
Stream                  

# saving all the high performance candidates to a separate data frame for further ranking within the group using z-score

In [162]:
dataset.groupby(['cluster_type']).count()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Performance_10,Degree_code,Stream_code,relative_count_skills,working_years,City_binned,performance_pg_percentage,performance_ug_percentage,performance_std10_percentage,performance_std12_percentage
cluster_type,,,,,,,,,,,,,,,,,,,,,
0,173,173,173,173,173,173,173,173,173,173,...,173,173,173,173,173,173,173,173,173,173
1,219,219,219,219,219,219,219,219,219,219,...,219,219,219,219,219,219,219,219,219,219


In [ ]:
#there are 173 candidates belonging to cluster 0,the first cluster,the high performing one.

In [164]:
filt=(dataset['cluster_type']==0)
dataset.loc[filt,:]

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Degree_code,Stream_code,relative_count_skills,working_years,City_binned,performance_pg_percentage,performance_ug_percentage,performance_std10_percentage,performance_std12_percentage,cluster_type
0,ML0001,Bangalore,0,2,0,2,0,2,3,2,...,bachelors program,2,0.00625,0,2,0.6885,0.6139,0.75165,0.7894,0
1,ML0002,Bangalore,2,0,0,2,2,2,2,2,...,bachelors program,2,0.03125,0,2,0.6550,0.8550,0.96600,0.9220,0
2,ML0003,Bangalore,3,0,1,2,2,2,0,2,...,post_graduate_program,2,0.10625,0,2,0.7910,0.7000,0.77600,0.6483,0
5,ML0006,Bangalore,2,0,0,1,0,3,2,1,...,bachelors program,2,0.02500,0,2,0.7780,0.7000,0.79600,0.6550,0
8,ML0009,Bangalore,3,0,0,0,0,2,2,0,...,post_graduate_program,2,0.13750,0,2,0.8000,0.7000,0.93000,0.7000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,ML0379,Shillong,2,0,0,2,3,3,2,2,...,post_graduate_program,2,0.08750,0,1,0.6800,0.7670,0.69560,0.7000,0
381,ML0382,Bangalore,2,0,0,3,0,3,2,3,...,post_graduate_program,2,0.07500,0,2,0.7160,0.8060,0.68000,0.5380,0
386,ML0387,Durg,1,1,0,2,2,2,2,2,...,bachelors program,2,0.11875,0,1,0.6530,0.7550,0.47000,0.6790,0
388,ML0389,Bangalore,2,0,0,2,0,2,2,1,...,post_graduate_program,2,0.04375,0,2,0.7780,0.6800,0.50650,0.9395,0


In [166]:
#high_dataset=normalized_final_df.loc[filt,:]

In [169]:
normalized_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 22 columns):
City_binned                     392 non-null float64
Python (out of 3)               392 non-null float64
R Programming (out of 3)        392 non-null float64
Deep Learning (out of 3)        392 non-null float64
PHP (out of 3)                  392 non-null float64
MySQL (out of 3)                392 non-null float64
HTML (out of 3)                 392 non-null float64
CSS (out of 3)                  392 non-null float64
JavaScript (out of 3)           392 non-null float64
AJAX (out of 3)                 392 non-null float64
Bootstrap (out of 3)            392 non-null float64
MongoDB (out of 3)              392 non-null float64
Node.js (out of 3)              392 non-null float64
ReactJS (out of 3)              392 non-null float64
relative_count_skills           392 non-null float64
Degree_code                     392 non-null float64
Stream_code                     392 non-null fl

In [ ]:
#calculating Z-score for all columns

In [170]:
cols = list(normalized_final_df.columns)
normalized_final_df[cols]
for col in cols:
    col_zscore = col + '_zscore'
    normalized_final_df[col_zscore] = (normalized_final_df[col] - normalized_final_df[col].mean())/normalized_final_df[col].std(ddof=0)
normalized_final_df

,City_binned,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),AJAX (out of 3),...,Node.js (out of 3)_zscore,ReactJS (out of 3)_zscore,relative_count_skills_zscore,Degree_code_zscore,Stream_code_zscore,working_years_zscore,performance_pg_percentage_zscore,performance_ug_percentage_zscore,performance_std10_percentage_zscore,performance_std12_percentage_zscore
0,1.0,0.000000,0.666667,0.000000,0.666667,0.000000,0.666667,1.000000,0.666667,0.0,...,-0.216267,-0.358057,-1.302862,-0.732977,0.778818,-0.341389,-0.502498,-0.713828,0.626019,0.536864
1,1.0,0.666667,0.000000,0.000000,0.666667,0.666667,0.666667,0.666667,0.666667,0.0,...,-0.216267,-0.358057,-0.516228,-0.732977,0.778818,-0.341389,-0.831376,1.160018,1.311369,1.155636
2,1.0,1.000000,0.000000,0.333333,0.666667,0.666667,0.666667,0.000000,0.666667,0.0,...,-0.216267,-0.358057,1.843672,1.364300,0.778818,-0.341389,0.503770,-0.044653,0.703874,-0.121573
3,1.0,0.666667,0.000000,0.666667,0.333333,0.000000,0.666667,0.000000,0.000000,0.0,...,-0.216267,-0.358057,-0.122911,-0.732977,-1.283997,-0.341389,-0.203072,-0.153462,0.918735,0.399670
4,1.0,0.666667,0.000000,0.000000,0.666667,0.000000,0.666667,0.333333,0.333333,0.0,...,4.770591,3.652186,0.270405,-0.732977,-1.283997,-0.341389,0.705023,-0.526521,0.422508,-0.104307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,0.0,0.666667,0.333333,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.0,...,-0.216267,-0.358057,-0.712887,-0.732977,0.778818,-0.341389,1.267061,0.343951,-0.418394,0.593795
388,1.0,0.666667,0.000000,0.000000,0.666667,0.000000,0.666667,0.666667,0.333333,0.0,...,-0.216267,-0.358057,-0.122911,1.364300,0.778818,-0.341389,0.376146,-0.200094,-0.157810,1.237299
389,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.666667,0.666667,0.333333,0.0,...,-0.216267,-0.358057,-0.712887,-0.732977,-1.283997,-0.341389,-0.679208,-0.744139,1.004904,-0.224702
390,0.0,0.666667,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,-0.216267,-0.358057,-0.516228,-0.732977,0.778818,-0.341389,0.297608,-0.433257,0.668703,-0.230302


In [171]:
normalized_final_df.columns

Index(['City_binned', 'Python (out of 3)', 'R Programming (out of 3)',
       'Deep Learning (out of 3)', 'PHP (out of 3)', 'MySQL (out of 3)',
       'HTML (out of 3)', 'CSS (out of 3)', 'JavaScript (out of 3)',
       'AJAX (out of 3)', 'Bootstrap (out of 3)', 'MongoDB (out of 3)',
       'Node.js (out of 3)', 'ReactJS (out of 3)', 'relative_count_skills',
       'Degree_code', 'Stream_code', 'working_years',
       'performance_pg_percentage', 'performance_ug_percentage',
       'performance_std10_percentage', 'performance_std12_percentage',
       'City_binned_zscore', 'Python (out of 3)_zscore',
       'R Programming (out of 3)_zscore', 'Deep Learning (out of 3)_zscore',
       'PHP (out of 3)_zscore', 'MySQL (out of 3)_zscore',
       'HTML (out of 3)_zscore', 'CSS (out of 3)_zscore',
       'JavaScript (out of 3)_zscore', 'AJAX (out of 3)_zscore',
       'Bootstrap (out of 3)_zscore', 'MongoDB (out of 3)_zscore',
       'Node.js (out of 3)_zscore', 'ReactJS (out of 3)_zscore',
 

In [ ]:
#separate dataframe for z-scores alone

In [172]:
zscore_subset_dataset=normalized_final_df.loc[0:,['City_binned_zscore', 'Python (out of 3)_zscore',
       'R Programming (out of 3)_zscore', 'Deep Learning (out of 3)_zscore',
       'PHP (out of 3)_zscore', 'MySQL (out of 3)_zscore',
       'HTML (out of 3)_zscore', 'CSS (out of 3)_zscore',
       'JavaScript (out of 3)_zscore', 'AJAX (out of 3)_zscore',
       'Bootstrap (out of 3)_zscore', 'MongoDB (out of 3)_zscore',
       'Node.js (out of 3)_zscore', 'ReactJS (out of 3)_zscore',
       'relative_count_skills_zscore', 'Degree_code_zscore',
       'Stream_code_zscore', 'working_years_zscore',
       'performance_pg_percentage_zscore', 'performance_ug_percentage_zscore',
       'performance_std10_percentage_zscore',
       'performance_std12_percentage_zscore']]

In [174]:
zscore_subset_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 22 columns):
City_binned_zscore                     392 non-null float64
Python (out of 3)_zscore               392 non-null float64
R Programming (out of 3)_zscore        392 non-null float64
Deep Learning (out of 3)_zscore        392 non-null float64
PHP (out of 3)_zscore                  392 non-null float64
MySQL (out of 3)_zscore                392 non-null float64
HTML (out of 3)_zscore                 392 non-null float64
CSS (out of 3)_zscore                  392 non-null float64
JavaScript (out of 3)_zscore           392 non-null float64
AJAX (out of 3)_zscore                 392 non-null float64
Bootstrap (out of 3)_zscore            392 non-null float64
MongoDB (out of 3)_zscore              392 non-null float64
Node.js (out of 3)_zscore              392 non-null float64
ReactJS (out of 3)_zscore              392 non-null float64
relative_count_skills_zscore           392 non-null flo

In [176]:
zscore_subset_dataset['total_zscore_all_features']=zscore_subset_dataset.sum(axis=1)

# Create a new column with z-scores
# ranked in descending order

In [180]:
zscore_subset_dataset["Rank"] = zscore_subset_dataset['total_zscore_all_features'].rank(ascending = 0)

In [183]:
s1 = zscore_subset_dataset["Rank"]
s2 = zscore_subset_dataset['total_zscore_all_features']
pd.concat([s1, s2], axis=1)

,Rank,total_zscore_all_features
0,79.0,5.256548
1,48.0,7.347921
2,23.0,10.308683
3,222.0,-1.296215
4,14.0,16.206003
...,...,...
387,266.0,-3.391892
388,86.5,4.871736
389,332.5,-7.023861
390,299.0,-5.441699


In [193]:
dataset['total_zscore_all_features']=zscore_subset_dataset['total_zscore_all_features']

# Adding rank to original dataset

In [184]:
dataset['rank']=zscore_subset_dataset["Rank"]

In [194]:
dataset.columns

Index(['Application_ID', 'Current City', 'Python (out of 3)',
       'R Programming (out of 3)', 'Deep Learning (out of 3)',
       'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)',
       'CSS (out of 3)', 'JavaScript (out of 3)', 'AJAX (out of 3)',
       'Bootstrap (out of 3)', 'MongoDB (out of 3)', 'Node.js (out of 3)',
       'ReactJS (out of 3)', 'Other skills', 'Degree', 'Stream',
       'Current Year Of Graduation', 'Performance_PG', 'Performance_UG',
       'Performance_12', 'Performance_10', 'Degree_code', 'Stream_code',
       'relative_count_skills', 'working_years', 'City_binned',
       'performance_pg_percentage', 'performance_ug_percentage',
       'performance_std10_percentage', 'performance_std12_percentage',
       'cluster_type', 'rank', 'total_zscore_all_features'],
      dtype='object')

In [195]:
filt=(dataset['rank']<11)
top_candidates_df=dataset.loc[filt,['Application_ID', 'Current City', 'Python (out of 3)',
       'R Programming (out of 3)', 'Deep Learning (out of 3)',
       'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)',
       'CSS (out of 3)', 'JavaScript (out of 3)', 'AJAX (out of 3)',
       'Bootstrap (out of 3)', 'MongoDB (out of 3)', 'Node.js (out of 3)',
       'ReactJS (out of 3)', 'Other skills', 'Degree', 'Stream',
       'Current Year Of Graduation', 'Performance_PG', 'Performance_UG',
       'Performance_12', 'Performance_10', 'Degree_code', 'Stream_code',
       'relative_count_skills', 'working_years', 'City_binned',
       'performance_pg_percentage', 'performance_ug_percentage',
       'performance_std10_percentage', 'performance_std12_percentage',
       'rank','total_zscore_all_features']]

In [196]:
top_candidates_df

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Stream_code,relative_count_skills,working_years,City_binned,performance_pg_percentage,performance_ug_percentage,performance_std10_percentage,performance_std12_percentage,rank,total_zscore_all_features
20,ML0021,Thrissur,1,0,0,2,2,3,3,2,...,2,0.04375,0,1,0.760000,0.8300,0.9130,0.7890,4.0,24.373301
38,ML0039,Bangalore,1,0,0,2,3,3,2,2,...,2,0.06250,0,2,0.680000,0.8500,0.8700,0.6530,8.0,17.960115
124,ML0125,Bangalore,3,2,3,2,2,2,2,2,...,2,0.04375,0,2,0.760000,0.7600,0.8233,0.6000,10.0,17.682531
139,ML0140,Chennai,2,0,2,2,2,3,3,2,...,1,0.03750,0,2,0.760000,0.6770,0.9560,0.9250,3.0,25.027483
182,ML0183,Bangalore,2,2,2,2,3,3,2,2,...,2,0.12500,0,2,0.680000,0.7435,0.7378,0.7405,5.0,22.604051
206,ML0207,Vellore,2,0,0,2,3,3,2,2,...,2,0.03125,2,1,0.680000,0.9300,0.6956,0.7000,7.0,18.414276
224,ML0225,Tezpur,2,0,0,2,0,2,1,1,...,1,0.08750,0,1,0.823571,0.7900,0.7140,0.9240,9.0,17.817718
240,ML0241,Bangalore,1,0,0,2,2,3,3,2,...,2,0.16875,0,2,0.760000,0.8300,0.9130,0.7890,2.0,30.330316
344,ML0345,Bangalore,2,2,3,2,2,2,2,2,...,2,0.14375,0,2,0.760000,0.7200,0.7800,0.7810,6.0,20.197664
359,ML0360,Bangalore,1,0,2,2,2,3,3,2,...,2,0.16875,0,2,0.760000,0.8300,0.9130,0.7890,1.0,32.707701


In [197]:
top_candidates_df.to_excel("Final_selection.xlsx")